# 📊 Análise Exploratória Automatizada - Framework Parametrizável

## Objetivo
Este notebook realiza uma **análise exploratória completa e automatizada** de qualquer tabela Delta Lake no Microsoft Fabric.

### Características:
- ✅ **100% Parametrizável** - Basta alterar os parâmetros na primeira célula
- ✅ **Análise em Massa** - Processa todas as colunas automaticamente
- ✅ **Detecção Automática de Tipos** - Adapta análises conforme o tipo de dado
- ✅ **Seguro contra Travamentos** - Limita cardinalidade e usa amostragem inteligente
- ✅ **Exportação de Resultados** - Gera relatórios consolidados
- ✅ **Metadados Enriquecidos** - Gera JSON para alimentar o Book de Variáveis

---
**Versão:** 2.1 | **Última Atualização:** Janeiro 2025

---
## 🎯 SEÇÃO 1: PARÂMETROS DE CONFIGURAÇÃO

⚠️ **ALTERE APENAS ESTA CÉLULA PARA RODAR EM OUTRAS TABELAS**

In [1]:
# === CONFIGURAÇÃO DO LAKEHOUSE ===
LAKEHOUSE_PATH = "abfss://febb8631-d5c0-43d8-bf08-5e89c8f2d17e@onelake.dfs.fabric.microsoft.com/5f8a4808-6f65-401b-a427-b0dd9d331b35" #SILVER
# LAKEHOUSE_PATH = "abfss://febb8631-d5c0-43d8-bf08-5e89c8f2d17e@onelake.dfs.fabric.microsoft.com/6a7135c7-0d8d-4625-815d-c4c4a02e4ed4" #GOLD

# === TABELA A SER ANALISADA ===
NOME_SCHEMA = "rawdata"
NOME_TABELA = "dados_cadastrais"
# NOME_SCHEMA = "feature_store"
# NOME_TABELA = "clientes_consolidado"

# === CAMINHOS DERIVADOS ===
PATH_INPUT = f"{LAKEHOUSE_PATH}/Tables/{NOME_SCHEMA}/{NOME_TABELA}"
PATH_OUTPUT = f"/lakehouse/default/Files/Metadados"

# === PARÂMETROS DE ANÁLISE ===
AMOSTRA_PERCENTUAL = 0.1
AMOSTRA_MAX_LINHAS = 1000000
CARDINALIDADE_MAX = 100
TOP_N_VALORES = 20

# === FLAGS DE CONTROLE ===
EXECUTAR_CORRELACOES = True
SALVAR_RESULTADOS = True
MODO_DEBUG = False

print(f"Configuracao: {NOME_SCHEMA}.{NOME_TABELA}")
print(f"Input: {PATH_INPUT}")
print(f"Output: {PATH_OUTPUT}")

StatementMeta(, f62946fa-a604-4d89-a172-4db426cf81a6, 3, Finished, Available, Finished)

Configuracao: rawdata.dados_cadastrais
Input: abfss://febb8631-d5c0-43d8-bf08-5e89c8f2d17e@onelake.dfs.fabric.microsoft.com/5f8a4808-6f65-401b-a427-b0dd9d331b35/Tables/rawdata/dados_cadastrais
Output: /lakehouse/default/Files/Metadados


---
## 🔧 SEÇÃO 2: SETUP E IMPORTAÇÕES

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from datetime import datetime
import os
import json
import warnings
warnings.filterwarnings('ignore')

base_path = "/lakehouse/default/Files/Metadados"
os.makedirs(base_path, exist_ok=True)

try:
    spark
    print("SparkSession ja existe")
except NameError:
    spark = SparkSession.builder.appName(f"EDA_{NOME_TABELA}").getOrCreate()
    print("SparkSession criada")

spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", "true")

TIMESTAMP_EXECUCAO = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Execucao iniciada em: {TIMESTAMP_EXECUCAO}")

StatementMeta(, f62946fa-a604-4d89-a172-4db426cf81a6, 4, Finished, Available, Finished)

SparkSession ja existe
Execucao iniciada em: 2026-01-22 23:09:54


---
## 📥 SEÇÃO 3: CARREGAMENTO DOS DADOS

In [3]:
print(f"Carregando tabela: {NOME_SCHEMA}.{NOME_TABELA}")
print("="*80)

try:
    df_raw = spark.read.format("delta").load(PATH_INPUT)
    print("Tabela carregada - Delta Lake")
except:
    try:
        df_raw = spark.read.parquet(PATH_INPUT)
        print("Tabela carregada - Parquet")
    except:
        df_raw = spark.table(f"{NOME_SCHEMA}.{NOME_TABELA}")
        print("Tabela carregada - Catalogo")

total_registros = df_raw.count()
total_colunas = len(df_raw.columns)

print(f"Total de Registros: {total_registros:,}")
print(f"Total de Colunas: {total_colunas}")

if total_registros > AMOSTRA_MAX_LINHAS:
    fracao = min(AMOSTRA_PERCENTUAL, AMOSTRA_MAX_LINHAS / total_registros)
    df = df_raw.sample(fraction=fracao, seed=42)
    df.cache()
    tamanho_amostra = df.count()
    print(f"AMOSTRAGEM: {fracao*100:.2f}% = {tamanho_amostra:,} registros")
else:
    df = df_raw
    df.cache()
    tamanho_amostra = total_registros
    print("Usando dados completos")

df.createOrReplaceTempView("tabela_analise")

StatementMeta(, f62946fa-a604-4d89-a172-4db426cf81a6, 5, Finished, Available, Finished)

Carregando tabela: rawdata.dados_cadastrais
Tabela carregada - Delta Lake
Total de Registros: 3,900,378
Total de Colunas: 36
AMOSTRAGEM: 10.00% = 389,941 registros


---
## 📋 SEÇÃO 4: SCHEMA E CLASSIFICAÇÃO DE COLUNAS

In [4]:
print("SCHEMA DA TABELA")
print("="*80)

schema_info = [{"coluna": f.name, "tipo_spark": str(f.dataType), "nullable": f.nullable} for f in df.schema.fields]
df_schema = spark.createDataFrame(schema_info)
df_schema.show(100, truncate=False)

colunas_numericas = []
colunas_string = []
colunas_data = []
colunas_booleanas = []
colunas_outras = []

for field in df.schema.fields:
    nome = field.name
    tipo = field.dataType
    if isinstance(tipo, (IntegerType, LongType, FloatType, DoubleType, DecimalType, ShortType, ByteType)):
        colunas_numericas.append(nome)
    elif isinstance(tipo, StringType):
        colunas_string.append(nome)
    elif isinstance(tipo, (DateType, TimestampType)):
        colunas_data.append(nome)
    elif isinstance(tipo, BooleanType):
        colunas_booleanas.append(nome)
    else:
        colunas_outras.append(nome)

print(f"\nNumericas ({len(colunas_numericas)}): {colunas_numericas[:5]}")
print(f"String ({len(colunas_string)}): {colunas_string[:5]}")
print(f"Data/Timestamp ({len(colunas_data)}): {colunas_data[:5]}")
print(f"Booleanas ({len(colunas_booleanas)}): {colunas_booleanas}")
print(f"Outras ({len(colunas_outras)}): {colunas_outras}")

StatementMeta(, f62946fa-a604-4d89-a172-4db426cf81a6, 6, Finished, Available, Finished)

SCHEMA DA TABELA
+----------------------+--------+---------------+
|coluna                |nullable|tipo_spark     |
+----------------------+--------+---------------+
|NUM_CPF               |true    |StringType()   |
|SAFRA                 |true    |IntegerType()  |
|FLAG_INSTALACAO       |true    |IntegerType()  |
|FPD                   |true    |IntegerType()  |
|PROD                  |true    |StringType()   |
|flag_mig2             |true    |StringType()   |
|STATUSRF              |true    |StringType()   |
|DATADENASCIMENTO      |true    |DateType()     |
|var_03                |true    |IntegerType()  |
|var_02                |true    |IntegerType()  |
|var_04                |true    |IntegerType()  |
|var_05                |true    |IntegerType()  |
|var_06                |true    |IntegerType()  |
|var_07                |true    |IntegerType()  |
|var_08                |true    |IntegerType()  |
|var_09                |true    |IntegerType()  |
|var_10                |true    |

---
## 🔍 SEÇÃO 5: ANÁLISE DE QUALIDADE DE DADOS

In [5]:
print("ANALISE DE QUALIDADE - COMPLETUDE")
print("="*80)

null_exprs = [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns]
null_counts = df.select(null_exprs).collect()[0]

qualidade_dados = []
for col_name in df.columns:
    nulls = null_counts[col_name]
    pct_null = (nulls / tamanho_amostra) * 100 if tamanho_amostra > 0 else 0
    pct_preenchido = 100 - pct_null
    if pct_preenchido >= 99: status = "EXCELENTE"
    elif pct_preenchido >= 90: status = "BOM"
    elif pct_preenchido >= 70: status = "ATENCAO"
    else: status = "CRITICO"
    qualidade_dados.append({"coluna": col_name, "nulls": nulls, "pct_null": round(pct_null, 2), "pct_preenchido": round(pct_preenchido, 2), "status": status})

df_qualidade = spark.createDataFrame(qualidade_dados).orderBy(F.desc("pct_null"))
print("RANKING POR % DE NULLS:")
df_qualidade.show(20, truncate=False)

colunas_criticas = df_qualidade.filter(F.col("pct_null") > 30).count()
colunas_perfeitas = df_qualidade.filter(F.col("pct_null") == 0).count()
print(f"Colunas 100% preenchidas: {colunas_perfeitas}")
print(f"Colunas criticas (>30% null): {colunas_criticas}")

StatementMeta(, f62946fa-a604-4d89-a172-4db426cf81a6, 7, Finished, Available, Finished)

ANALISE DE QUALIDADE - COMPLETUDE
RANKING POR % DE NULLS:
+----------------+------+--------+--------------+-------+
|coluna          |nulls |pct_null|pct_preenchido|status |
+----------------+------+--------+--------------+-------+
|DATADENASCIMENTO|389941|100.0   |0.0           |CRITICO|
|var_11          |384128|98.51   |1.49          |CRITICO|
|var_10          |383702|98.4    |1.6           |CRITICO|
|var_20          |382694|98.14   |1.86          |CRITICO|
|var_02          |368423|94.48   |5.52          |CRITICO|
|var_22          |353221|90.58   |9.42          |CRITICO|
|var_14          |353221|90.58   |9.42          |CRITICO|
|var_23          |331700|85.06   |14.94         |CRITICO|
|var_16          |331700|85.06   |14.94         |CRITICO|
|var_15          |331700|85.06   |14.94         |CRITICO|
|var_17          |331700|85.06   |14.94         |CRITICO|
|var_13          |331159|84.93   |15.07         |CRITICO|
|var_07          |325058|83.36   |16.64         |CRITICO|
|var_18       

In [6]:
print("ANALISE DE CARDINALIDADE")
print("="*80)

distinct_exprs = [F.countDistinct(F.col(c)).alias(c) for c in df.columns]
distinct_counts = df.select(distinct_exprs).collect()[0]

cardinalidade_dados = []
for col_name in df.columns:
    distintos = distinct_counts[col_name]
    taxa_unicidade = (distintos / tamanho_amostra) * 100 if tamanho_amostra > 0 else 0
    if distintos == 1: tipo_card = "CONSTANTE"
    elif distintos <= 10: tipo_card = "BAIXA (Flag/Status)"
    elif distintos <= 100: tipo_card = "MEDIA (Categorica)"
    elif taxa_unicidade > 95: tipo_card = "MUITO ALTA (PK/ID)"
    else: tipo_card = "ALTA"
    cardinalidade_dados.append({"coluna": col_name, "valores_unicos": distintos, "taxa_unicidade_pct": round(taxa_unicidade, 2), "tipo_cardinalidade": tipo_card})

df_cardinalidade = spark.createDataFrame(cardinalidade_dados).orderBy("valores_unicos")
df_cardinalidade.show(100, truncate=False)

potenciais_pks = [r["coluna"] for r in cardinalidade_dados if r["taxa_unicidade_pct"] > 99]
colunas_constantes = [r["coluna"] for r in cardinalidade_dados if r["valores_unicos"] == 1]
print(f"Potenciais PKs: {potenciais_pks}")
print(f"Colunas constantes: {colunas_constantes}")

StatementMeta(, f62946fa-a604-4d89-a172-4db426cf81a6, 8, Finished, Available, Finished)

ANALISE DE CARDINALIDADE
+----------------------+------------------+-------------------+--------------+
|coluna                |taxa_unicidade_pct|tipo_cardinalidade |valores_unicos|
+----------------------+------------------+-------------------+--------------+
|DATADENASCIMENTO      |0.0               |BAIXA (Flag/Status)|0             |
|var_06                |0.0               |CONSTANTE          |1             |
|var_20                |0.0               |CONSTANTE          |1             |
|var_18                |0.0               |CONSTANTE          |1             |
|_execution_id         |0.0               |CONSTANTE          |1             |
|var_19                |0.0               |CONSTANTE          |1             |
|var_22                |0.0               |CONSTANTE          |1             |
|_data_inclusao        |0.0               |CONSTANTE          |1             |
|var_21                |0.0               |CONSTANTE          |1             |
|_data_alteracao_silver|0.0

---
## 📈 SEÇÃO 6: ANÁLISE DE COLUNAS NUMÉRICAS

In [7]:
print("ANALISE DE COLUNAS NUMERICAS")
print("="*80)

def normalizar_stats(stats):
    """
    Garante que valores numericos sejam coerentes:
    - ints -> float se a coluna tiver algum float
    - None preservado
    """
    from collections import defaultdict

    tipos = defaultdict(set)

    # 1) Detecta tipos por coluna
    for row in stats:
        for k, v in row.items():
            if v is not None:
                tipos[k].add(type(v))

    # 2) Define colunas que devem virar float
    colunas_float = {
        k for k, t in tipos.items()
        if float in t or any(issubclass(x, float) for x in t)
    }

    # 3) Normaliza valores
    normalizado = []
    for row in stats:
        novo = {}
        for k, v in row.items():
            if v is None:
                novo[k] = None
            elif k in colunas_float:
                novo[k] = float(v)
            else:
                novo[k] = int(v) if isinstance(v, int) else v
        normalizado.append(novo)

    return normalizado

stats_numericas = []
if colunas_numericas:
    print(f"Analisando {len(colunas_numericas)} colunas numericas...")
    df.select(colunas_numericas).describe().show(truncate=False)
    
    for col_name in colunas_numericas:
        try:
            stats = df.select(
                F.count(col_name).alias("count"),
                F.sum(col_name).alias("soma"),
                F.avg(col_name).alias("media"),
                F.stddev(col_name).alias("desvio_padrao"),
                F.min(col_name).alias("minimo"),
                F.max(col_name).alias("maximo"),
                F.expr(f"percentile_approx({col_name}, 0.25)").alias("p25"),
                F.expr(f"percentile_approx({col_name}, 0.50)").alias("mediana"),
                F.expr(f"percentile_approx({col_name}, 0.75)").alias("p75"),
                F.expr(f"percentile_approx({col_name}, 0.95)").alias("p95"),
                F.expr(f"percentile_approx({col_name}, 0.99)").alias("p99"),
                F.sum(F.when(F.col(col_name) == 0, 1).otherwise(0)).alias("qtd_zeros"),
                F.sum(F.when(F.col(col_name) < 0, 1).otherwise(0)).alias("qtd_negativos")
            ).collect()[0]
            
            p25 = float(stats["p25"]) if stats["p25"] is not None else 0
            p75 = float(stats["p75"]) if stats["p75"] is not None else 0
            iqr = p75 - p25
            
            stats_numericas.append({
                "coluna": col_name,
                "count": stats["count"],
                "soma": float(stats["soma"]) if stats["soma"] else None,
                "media": float(stats["media"]) if stats["media"] else None,
                "desvio_padrao": float(stats["desvio_padrao"]) if stats["desvio_padrao"] else None,
                "minimo": float(stats["minimo"]) if stats["minimo"] else None,
                "p25": p25,
                "mediana": float(stats["mediana"]) if stats["mediana"] else None,
                "p75": p75,
                "p95": float(stats["p95"]) if stats["p95"] else None,
                "p99": float(stats["p99"]) if stats["p99"] else None,
                "maximo": float(stats["maximo"]) if stats["maximo"] else None,
                "iqr": iqr,
                "limite_outlier_inf": p25 - 1.5 * iqr,
                "limite_outlier_sup": p75 + 1.5 * iqr,
                "qtd_zeros": stats["qtd_zeros"],
                "qtd_negativos": stats["qtd_negativos"]
            })
        except Exception as e:
            if MODO_DEBUG: print(f"Erro em {col_name}: {str(e)[:50]}")
    
    if stats_numericas:
        df_stats_num = spark.createDataFrame(
            normalizar_stats(stats_numericas)
        )
        print("\nESTATISTICAS AVANCADAS:")
        df_stats_num.show(100, truncate=False)
else:
    print("Nenhuma coluna numerica encontrada.")

StatementMeta(, f62946fa-a604-4d89-a172-4db426cf81a6, 9, Finished, Available, Finished)

ANALISE DE COLUNAS NUMERICAS
Analisando 14 colunas numericas...
+-------+------------------+-------------------+-------------------+------------------+------------------+-------------------+------------------+------+------------------+------------------+------------------+------------------+------------------+------------------+
|summary|SAFRA             |FLAG_INSTALACAO    |FPD                |var_03            |var_02            |var_04             |var_05            |var_06|var_07            |var_08            |var_09            |var_14            |var_16            |var_17            |
+-------+------------------+-------------------+-------------------+------------------+------------------+-------------------+------------------+------+------------------+------------------+------------------+------------------+------------------+------------------+
|count  |389941            |389941             |269694             |362790            |21518             |388374             |370262   

In [8]:
print("DETECCAO DE OUTLIERS (Metodo IQR)")
print("="*80)

if colunas_numericas and stats_numericas:
    outliers_resumo = []
    for stat in stats_numericas:
        col_name = stat["coluna"]
        lim_inf = stat["limite_outlier_inf"]
        lim_sup = stat["limite_outlier_sup"]
        if lim_inf is not None and lim_sup is not None:
            outliers_count = df.filter((F.col(col_name) < lim_inf) | (F.col(col_name) > lim_sup)).count()
            pct_outliers = (outliers_count / tamanho_amostra) * 100 if tamanho_amostra > 0 else 0
            outliers_resumo.append({"coluna": col_name, "limite_inferior": round(lim_inf, 4), "limite_superior": round(lim_sup, 4), "qtd_outliers": outliers_count, "pct_outliers": round(pct_outliers, 2)})
    
    if outliers_resumo:
        df_outliers = spark.createDataFrame(outliers_resumo).orderBy(F.desc("pct_outliers"))
        df_outliers.show(50, truncate=False)
else:
    print("Analise de outliers nao disponivel.")

StatementMeta(, f62946fa-a604-4d89-a172-4db426cf81a6, 10, Finished, Available, Finished)

DETECCAO DE OUTLIERS (Metodo IQR)
+---------------+---------------+---------------+------------+------------+
|coluna         |limite_inferior|limite_superior|pct_outliers|qtd_outliers|
+---------------+---------------+---------------+------------+------------+
|FPD            |0.0            |0.0            |14.63       |57059       |
|var_04         |0.0            |0.0            |10.7        |41730       |
|var_03         |-32.0          |80.0           |10.08       |39308       |
|var_05         |-0.5           |3.5            |9.77        |38097       |
|var_17         |202168.5       |202548.5       |2.45        |9536        |
|var_09         |4.0            |12.0           |1.73        |6728        |
|var_14         |1.0            |1.0            |1.08        |4194        |
|var_07         |-90830.0       |154018.0       |0.72        |2791        |
|var_16         |-132.5         |1263.5         |0.13        |518         |
|var_08         |-42.5          |153.5          |0.0  

---
## 📝 SEÇÃO 7: ANÁLISE DE COLUNAS CATEGÓRICAS

In [9]:
print("ANALISE DE COLUNAS STRING/CATEGORICAS")
print("="*80)

colunas_categoricas = []
colunas_texto_livre = []

if colunas_string:
    for row in cardinalidade_dados:
        if row["coluna"] in colunas_string:
            if row["valores_unicos"] <= CARDINALIDADE_MAX:
                colunas_categoricas.append(row["coluna"])
            else:
                colunas_texto_livre.append(row["coluna"])
    
    print(f"Categoricas (<={CARDINALIDADE_MAX} valores): {len(colunas_categoricas)}")
    print(f"Texto Livre (>{CARDINALIDADE_MAX} valores): {len(colunas_texto_livre)}")
    
    print("\nDISTRIBUICAO DE VALORES:")
    for col_name in colunas_categoricas:
        print(f"\n--- {col_name} ---")
        df.groupBy(col_name).agg(
            F.count("*").alias("frequencia"),
            F.round(F.count("*") / tamanho_amostra * 100, 2).alias("percentual")
        ).orderBy(F.desc("frequencia")).limit(TOP_N_VALORES).show(truncate=False)
else:
    print("Nenhuma coluna string encontrada.")

StatementMeta(, f62946fa-a604-4d89-a172-4db426cf81a6, 11, Finished, Available, Finished)

ANALISE DE COLUNAS STRING/CATEGORICAS
Categoricas (<=100 valores): 13
Texto Livre (>100 valores): 6

DISTRIBUICAO DE VALORES:

--- PROD ---
+----+----------+----------+
|PROD|frequencia|percentual|
+----+----------+----------+
|CMV |379429    |97.3      |
|NET |9003      |2.31      |
|DTH |1509      |0.39      |
+----+----------+----------+


--- flag_mig2 ---
+---------+----------+----------+
|flag_mig2|frequencia|percentual|
+---------+----------+----------+
|Aquisição|134116    |34.39     |
|PRE      |128855    |33.04     |
|NULL     |126524    |32.45     |
|FLEX     |446       |0.11      |
+---------+----------+----------+


--- STATUSRF ---
+-------------------------+----------+----------+
|STATUSRF                 |frequencia|percentual|
+-------------------------+----------+----------+
|REGULAR                  |384752    |98.67     |
|PENDENTE DE REGULARIZACAO|3116      |0.8       |
|NULL                     |1567      |0.4       |
|SUSPENSA                 |247       |0.06    

In [10]:
print("PADROES EM COLUNAS STRING")
print("="*80)

if colunas_string:
    padroes_string = []
    for col_name in colunas_string:
        try:
            stats = df.select(
                F.count(col_name).alias("count_nao_null"),
                F.avg(F.length(col_name)).alias("comprimento_medio"),
                F.min(F.length(col_name)).alias("comprimento_min"),
                F.max(F.length(col_name)).alias("comprimento_max"),
                F.sum(F.when(F.col(col_name) == "", 1).otherwise(0)).alias("qtd_vazios"),
                F.sum(F.when(F.col(col_name).rlike("^[0-9]+$"), 1).otherwise(0)).alias("qtd_somente_numeros"),
                F.sum(F.when(F.col(col_name).rlike("^[A-Za-z]+$"), 1).otherwise(0)).alias("qtd_somente_letras")
            ).collect()[0]
            padroes_string.append({
                "coluna": col_name,
                "count_nao_null": stats["count_nao_null"],
                "comprimento_medio": round(float(stats["comprimento_medio"]), 2) if stats["comprimento_medio"] else None,
                "comprimento_min": stats["comprimento_min"],
                "comprimento_max": stats["comprimento_max"],
                "qtd_vazios": stats["qtd_vazios"],
                "qtd_somente_numeros": stats["qtd_somente_numeros"],
                "qtd_somente_letras": stats["qtd_somente_letras"]
            })
        except: pass
    
    if padroes_string:
        spark.createDataFrame(padroes_string).show(100, truncate=False)

StatementMeta(, f62946fa-a604-4d89-a172-4db426cf81a6, 12, Finished, Available, Finished)

PADROES EM COLUNAS STRING
+-------------+---------------+-----------------+---------------+--------------+------------------+-------------------+----------+
|coluna       |comprimento_max|comprimento_medio|comprimento_min|count_nao_null|qtd_somente_letras|qtd_somente_numeros|qtd_vazios|
+-------------+---------------+-----------------+---------------+--------------+------------------+-------------------+----------+
|NUM_CPF      |11             |11.0             |11             |389941        |8909              |0                  |0         |
|PROD         |3              |3.0              |3              |389941        |389941            |0                  |0         |
|flag_mig2    |9              |6.06             |3              |263417        |129301            |0                  |0         |
|STATUSRF     |25             |7.15             |4              |388374        |385020            |0                  |0         |
|var_10       |57             |20.31            |2       

---
## 📅 SEÇÃO 8: ANÁLISE DE COLUNAS TEMPORAIS

In [11]:
print("ANALISE DE COLUNAS TEMPORAIS")
print("="*80)

if colunas_data:
    print(f"Analisando {len(colunas_data)} colunas temporais...")
    stats_temporais = []
    for col_name in colunas_data:
        try:
            stats = df.select(
                F.count(col_name).alias("count_nao_null"),
                F.min(col_name).alias("data_minima"),
                F.max(col_name).alias("data_maxima"),
                F.countDistinct(F.date_format(col_name, "yyyy-MM")).alias("meses_distintos"),
                F.countDistinct(F.date_format(col_name, "yyyy")).alias("anos_distintos")
            ).collect()[0]
            amplitude = df.select(F.datediff(F.max(col_name), F.min(col_name)).alias("dias")).collect()[0]["dias"] if stats["data_minima"] and stats["data_maxima"] else None
            stats_temporais.append({
                "coluna": col_name,
                "count_nao_null": stats["count_nao_null"],
                "data_minima": str(stats["data_minima"]) if stats["data_minima"] else None,
                "data_maxima": str(stats["data_maxima"]) if stats["data_maxima"] else None,
                "amplitude_dias": amplitude,
                "meses_distintos": stats["meses_distintos"],
                "anos_distintos": stats["anos_distintos"]
            })
        except: pass
    
    if stats_temporais:
        spark.createDataFrame(stats_temporais).show(50, truncate=False)
        print(f"\nDistribuicao mensal - {colunas_data[0]}:")
        df.groupBy(F.date_format(colunas_data[0], "yyyy-MM").alias("ano_mes")).agg(F.count("*").alias("quantidade")).orderBy("ano_mes").show(24)
else:
    print("Nenhuma coluna temporal encontrada.")

StatementMeta(, f62946fa-a604-4d89-a172-4db426cf81a6, 13, Finished, Available, Finished)

ANALISE DE COLUNAS TEMPORAIS
Analisando 3 colunas temporais...
+--------------+--------------+----------------------+--------------+--------------------------+--------------------------+---------------+
|amplitude_dias|anos_distintos|coluna                |count_nao_null|data_maxima               |data_minima               |meses_distintos|
+--------------+--------------+----------------------+--------------+--------------------------+--------------------------+---------------+
|NULL          |0             |DATADENASCIMENTO      |0             |NULL                      |NULL                      |0              |
|0             |1             |_data_inclusao        |389941        |2026-01-12 19:13:12.075743|2026-01-12 19:13:12.075743|1              |
|0             |1             |_data_alteracao_silver|389941        |2026-01-14 03:30:20.323564|2026-01-14 03:30:20.323564|1              |
+--------------+--------------+----------------------+--------------+--------------------------+-

---
## 🔗 SEÇÃO 9: ANÁLISE DE CORRELAÇÕES

In [12]:
alta_corr = []
if EXECUTAR_CORRELACOES and len(colunas_numericas) >= 2:
    print("ANALISE DE CORRELACOES")
    print("="*80)
    from itertools import combinations
    cols_para_corr = colunas_numericas[:15]
    print(f"Calculando correlacoes para {len(cols_para_corr)} colunas...")
    correlacoes = []
    for col1, col2 in combinations(cols_para_corr, 2):
        try:
            corr_value = df.stat.corr(col1, col2)
            if corr_value is not None:
                correlacoes.append({"coluna_1": col1, "coluna_2": col2, "correlacao": round(corr_value, 4), "forca": "FORTE" if abs(corr_value) > 0.7 else "MODERADA" if abs(corr_value) > 0.4 else "FRACA"})
        except: pass
    if correlacoes:
        df_corr = spark.createDataFrame(correlacoes).orderBy(F.abs(F.col("correlacao")).desc())
        df_corr.show(30, truncate=False)
        alta_corr = [r for r in correlacoes if abs(r["correlacao"]) > 0.85]
        if alta_corr:
            print("ALERTA - Multicolinearidade (>0.85):")
            for p in alta_corr: print(f"  {p['coluna_1']} <-> {p['coluna_2']}: {p['correlacao']}")
else:
    print("Correlacoes desabilitadas ou insuficientes colunas numericas.")

StatementMeta(, f62946fa-a604-4d89-a172-4db426cf81a6, 14, Finished, Available, Finished)

ANALISE DE CORRELACOES
Calculando correlacoes para 14 colunas...
+---------------+--------+----------+-----+
|coluna_1       |coluna_2|correlacao|forca|
+---------------+--------+----------+-----+
|var_16         |var_17  |0.8976    |FORTE|
|var_06         |var_08  |0.8648    |FORTE|
|var_09         |var_17  |0.3674    |FRACA|
|var_09         |var_16  |0.3389    |FRACA|
|FLAG_INSTALACAO|FPD     |0.2765    |FRACA|
|var_05         |var_09  |-0.2257   |FRACA|
|var_04         |var_05  |0.2134    |FRACA|
|var_05         |var_17  |-0.17     |FRACA|
|var_05         |var_16  |-0.1613   |FRACA|
|FLAG_INSTALACAO|var_05  |0.1541    |FRACA|
|var_05         |var_14  |0.1455    |FRACA|
|var_06         |var_09  |-0.1449   |FRACA|
|var_06         |var_07  |0.1055    |FRACA|
|var_04         |var_14  |0.1051    |FRACA|
|var_08         |var_09  |-0.1      |FRACA|
|var_07         |var_08  |0.0898    |FRACA|
|var_04         |var_17  |-0.0874   |FRACA|
|FLAG_INSTALACAO|var_16  |-0.0856   |FRACA|
|FLAG_INSTA

---
## 🔎 SEÇÃO 10: ANÁLISE DE DUPLICATAS

In [13]:
print("ANALISE DE DUPLICATAS")
print("="*80)

total_linhas = df.count()
linhas_distintas = df.distinct().count()
duplicatas_exatas = total_linhas - linhas_distintas

print(f"Total de linhas: {total_linhas:,}")
print(f"Linhas distintas: {linhas_distintas:,}")
print(f"Duplicatas exatas: {duplicatas_exatas:,} ({duplicatas_exatas/total_linhas*100:.2f}%)")

if potenciais_pks:
    print("\nDuplicatas por potenciais PKs:")
    for pk in potenciais_pks[:5]:
        dup_pk = df.groupBy(pk).count().filter(F.col("count") > 1).count()
        print(f"  {pk}: {dup_pk:,} valores com duplicatas")

StatementMeta(, f62946fa-a604-4d89-a172-4db426cf81a6, 15, Finished, Available, Finished)

ANALISE DE DUPLICATAS
Total de linhas: 389,941
Linhas distintas: 389,941
Duplicatas exatas: 0 (0.00%)

Duplicatas por potenciais PKs:
  NUM_CPF: 3,318 valores com duplicatas


---
## 📊 SEÇÃO 11: RESUMO EXECUTIVO

In [14]:
print("\n" + "="*80)
print("                    RESUMO EXECUTIVO DA ANALISE")
print("="*80)
print(f"Tabela: {NOME_SCHEMA}.{NOME_TABELA}")
print(f"Data: {TIMESTAMP_EXECUCAO}")
print(f"Total Registros: {total_registros:,}")
print(f"Registros Analisados: {tamanho_amostra:,}")
print(f"Total Colunas: {total_colunas}")
print(f"\nDistribuicao de Tipos:")
print(f"  Numericas: {len(colunas_numericas)}")
print(f"  String: {len(colunas_string)}")
print(f"  Temporais: {len(colunas_data)}")
print(f"  Booleanas: {len(colunas_booleanas)}")
print(f"\nQualidade:")
print(f"  Colunas 100% preenchidas: {colunas_perfeitas}")
print(f"  Colunas criticas (>30% null): {colunas_criticas}")
print(f"  Colunas constantes: {len(colunas_constantes)}")
print(f"  Duplicatas exatas: {duplicatas_exatas:,}")
print(f"\nPotenciais PKs: {potenciais_pks[:3]}")

print("\nRECOMENDACOES:")
if colunas_constantes: print(f"  - Remover {len(colunas_constantes)} colunas constantes")
if colunas_criticas > 0: print(f"  - Investigar {colunas_criticas} colunas com >30% nulls")
if duplicatas_exatas > 0: print(f"  - Investigar {duplicatas_exatas:,} duplicatas")
if alta_corr: print(f"  - {len(alta_corr)} pares com multicolinearidade")
print("="*80)

StatementMeta(, f62946fa-a604-4d89-a172-4db426cf81a6, 16, Finished, Available, Finished)


                    RESUMO EXECUTIVO DA ANALISE
Tabela: rawdata.dados_cadastrais
Data: 2026-01-22 23:09:54
Total Registros: 3,900,378
Registros Analisados: 389,941
Total Colunas: 36

Distribuicao de Tipos:
  Numericas: 14
  String: 19
  Temporais: 3
  Booleanas: 0

Qualidade:
  Colunas 100% preenchidas: 7
  Colunas criticas (>30% null): 23
  Colunas constantes: 10
  Duplicatas exatas: 0

Potenciais PKs: ['NUM_CPF']

RECOMENDACOES:
  - Remover 10 colunas constantes
  - Investigar 23 colunas com >30% nulls
  - 2 pares com multicolinearidade


---
## 🧠 SEÇÃO 12: METADADOS ENRIQUECIDOS PARA BOOK DE VARIÁVEIS

Esta seção gera um DataFrame consolidado com todas as informações necessárias para criar um Book de Variáveis preciso.

In [15]:
print("GERANDO METADADOS ENRIQUECIDOS PARA BOOK")
print("="*80)

import json

metadados_enriquecidos = []

for field in df.schema.fields:
    col_name = field.name
    tipo_spark = str(field.dataType)
    
    info_qualidade = next((q for q in qualidade_dados if q['coluna'] == col_name), {})
    info_cardinalidade = next((c for c in cardinalidade_dados if c['coluna'] == col_name), {})
    
    pct_null = info_qualidade.get('pct_null', 0)
    cardinalidade = info_cardinalidade.get('valores_unicos', 0)
    taxa_unicidade = info_cardinalidade.get('taxa_unicidade_pct', 0)
    
    # Classificacao do tipo de variavel
    if col_name.startswith('_'):
        tipo_variavel = 'METADADO_SISTEMA'
        agregacoes = []
        usar_no_book = False
    elif taxa_unicidade > 95:
        tipo_variavel = 'IDENTIFICADOR_PK'
        agregacoes = ['COUNT_DISTINCT']
        usar_no_book = False
    elif col_name in colunas_numericas:
        tipo_variavel = 'NUMERICA_VALOR'
        agregacoes = ['SUM', 'AVG', 'MAX', 'MIN', 'STDDEV', 'COUNT']
        usar_no_book = True
    elif col_name in colunas_data:
        tipo_variavel = 'TEMPORAL'
        agregacoes = ['MIN', 'MAX', 'DATEDIFF', 'COUNT_DISTINCT_MES']
        usar_no_book = True
    elif col_name in colunas_string and cardinalidade <= 10:
        tipo_variavel = 'CATEGORICA_FLAG'
        agregacoes = ['TAXA_POR_VALOR', 'FLAG_TEM_VALOR', 'COUNT', 'MODE']
        usar_no_book = True
    elif col_name in colunas_string and cardinalidade <= CARDINALIDADE_MAX:
        tipo_variavel = 'CATEGORICA_MEDIA'
        agregacoes = ['COUNT_DISTINCT', 'MODE', 'TAXA_TOP_VALORES']
        usar_no_book = True
    elif col_name in colunas_string:
        tipo_variavel = 'TEXTO_LIVRE'
        agregacoes = ['COUNT_DISTINCT']
        usar_no_book = False
    elif col_name.startswith('DW_') or col_name.startswith('SK_'):
        tipo_variavel = 'CHAVE_DW'
        agregacoes = ['COUNT_DISTINCT']
        usar_no_book = False
    else:
        tipo_variavel = 'OUTROS'
        agregacoes = ['COUNT']
        usar_no_book = False
    
    # Qualidade para modelo
    if pct_null > 50:
        qualidade_modelo = 'BAIXA_MUITOS_NULLS'
        usar_no_book = False
    elif pct_null > 20:
        qualidade_modelo = 'MEDIA_REQUER_TRATAMENTO'
    elif cardinalidade == 1:
        qualidade_modelo = 'INUTILIZAVEL_CONSTANTE'
        usar_no_book = False
    else:
        qualidade_modelo = 'BOA'
    
    # Valores frequentes
    valores_frequentes = []
    if tipo_variavel in ['CATEGORICA_FLAG', 'CATEGORICA_MEDIA'] and cardinalidade <= 20:
        try:
            top_valores = df.groupBy(col_name).count().orderBy(F.desc('count')).limit(10).collect()
            valores_frequentes = [{'valor': str(row[col_name]), 'frequencia': row['count']} for row in top_valores]
        except: pass
    
    # Percentis
    percentis_valores = {}
    if tipo_variavel == 'NUMERICA_VALOR':
        info_stats = next((s for s in stats_numericas if s['coluna'] == col_name), {})
        if info_stats:
            percentis_valores = {
                'min': info_stats.get('minimo'),
                'p25': info_stats.get('p25'),
                'p50': info_stats.get('mediana'),
                'p75': info_stats.get('p75'),
                'p95': info_stats.get('p95'),
                'p99': info_stats.get('p99'),
                'max': info_stats.get('maximo'),
                'media': info_stats.get('media'),
                'desvio_padrao': info_stats.get('desvio_padrao')
            }
    
    # Sugestoes de variaveis
    sugestoes_variaveis = []
    if tipo_variavel == 'NUMERICA_VALOR':
        base_name = col_name.replace('VAL_', '').replace('NUM_', '').replace('QTD_', '')
        sugestoes_variaveis = [f"VLR_{base_name}_TOTAL", f"VLR_{base_name}_MEDIO", f"VLR_{base_name}_MAX", f"QTD_{base_name}"]
    elif tipo_variavel == 'CATEGORICA_FLAG':
        base_name = col_name.replace('IND_', '').replace('COD_', '').replace('STATUS_', '')
        sugestoes_variaveis = [f"TAXA_{base_name}_[VALOR]", f"FLAG_{base_name}_[VALOR]", f"QTD_{base_name}_DISTINTOS"]
    elif tipo_variavel == 'TEMPORAL':
        base_name = col_name.replace('DAT_', '').replace('DATA_', '')
        sugestoes_variaveis = [f"DIAS_DESDE_{base_name}", f"DIAS_ATE_{base_name}", f"MES_{base_name}"]
    
    metadados_enriquecidos.append({
        'coluna': col_name,
        'tipo_spark': tipo_spark,
        'tipo_variavel': tipo_variavel,
        'cardinalidade': cardinalidade,
        'pct_null': pct_null,
        'qualidade_modelo': qualidade_modelo,
        'usar_no_book': usar_no_book,
        'agregacoes_sugeridas': agregacoes,
        'valores_frequentes': valores_frequentes[:5],
        'percentis': percentis_valores,
        'sugestoes_variaveis': sugestoes_variaveis
    })

print(f"Metadados gerados para {len(metadados_enriquecidos)} colunas")

StatementMeta(, f62946fa-a604-4d89-a172-4db426cf81a6, 17, Finished, Available, Finished)

GERANDO METADADOS ENRIQUECIDOS PARA BOOK
Metadados gerados para 36 colunas


In [16]:
print("RESUMO DOS METADADOS ENRIQUECIDOS")
print("="*80)

from collections import Counter
contagem_tipos = Counter([m['tipo_variavel'] for m in metadados_enriquecidos])

print("\nDistribuicao por tipo de variavel:")
for tipo, qtd in sorted(contagem_tipos.items(), key=lambda x: -x[1]):
    usar = sum(1 for m in metadados_enriquecidos if m['tipo_variavel'] == tipo and m['usar_no_book'])
    print(f"  {tipo}: {qtd} colunas ({usar} utilizaveis no book)")

colunas_book = [m for m in metadados_enriquecidos if m['usar_no_book']]
print(f"\nCOLUNAS RECOMENDADAS PARA O BOOK: {len(colunas_book)} de {len(metadados_enriquecidos)}")

df_metadados_resumo = spark.createDataFrame([{
    'coluna': m['coluna'],
    'tipo_variavel': m['tipo_variavel'],
    'cardinalidade': m['cardinalidade'],
    'pct_null': m['pct_null'],
    'qualidade': m['qualidade_modelo'],
    'usar_book': 'SIM' if m['usar_no_book'] else 'NAO',
    'agregacoes': ', '.join(m['agregacoes_sugeridas'][:3])
} for m in metadados_enriquecidos])

df_metadados_resumo.show(100, truncate=False)

StatementMeta(, f62946fa-a604-4d89-a172-4db426cf81a6, 18, Finished, Available, Finished)

RESUMO DOS METADADOS ENRIQUECIDOS

Distribuicao por tipo de variavel:
  NUMERICA_VALOR: 14 colunas (6 utilizaveis no book)
  CATEGORICA_FLAG: 10 colunas (5 utilizaveis no book)
  TEXTO_LIVRE: 5 colunas (0 utilizaveis no book)
  METADADO_SISTEMA: 3 colunas (0 utilizaveis no book)
  CATEGORICA_MEDIA: 2 colunas (1 utilizaveis no book)
  IDENTIFICADOR_PK: 1 colunas (0 utilizaveis no book)
  TEMPORAL: 1 colunas (0 utilizaveis no book)

COLUNAS RECOMENDADAS PARA O BOOK: 12 de 36
+--------------------------------------+-------------+----------------------+--------+-----------------------+----------------+---------+
|agregacoes                            |cardinalidade|coluna                |pct_null|qualidade              |tipo_variavel   |usar_book|
+--------------------------------------+-------------+----------------------+--------+-----------------------+----------------+---------+
|COUNT_DISTINCT                        |386599       |NUM_CPF               |0.0     |BOA                   

In [17]:
print("DETALHAMENTO - COLUNAS CATEGORICAS")
print("="*80)

categoricas_com_valores = [m for m in metadados_enriquecidos if m['tipo_variavel'] in ['CATEGORICA_FLAG', 'CATEGORICA_MEDIA'] and m['valores_frequentes']]

for meta in categoricas_com_valores:
    print(f"\n{meta['coluna']} ({meta['tipo_variavel']})")
    print(f"  Cardinalidade: {meta['cardinalidade']} | Nulls: {meta['pct_null']}%")
    print("  Valores frequentes:")
    for v in meta['valores_frequentes']:
        pct = (v['frequencia'] / tamanho_amostra) * 100
        print(f"    '{v['valor']}': {v['frequencia']:,} ({pct:.1f}%)")
    print("  Sugestoes:")
    for sug in meta['sugestoes_variaveis']: print(f"    -> {sug}")

StatementMeta(, f62946fa-a604-4d89-a172-4db426cf81a6, 19, Finished, Available, Finished)

DETALHAMENTO - COLUNAS CATEGORICAS

PROD (CATEGORICA_FLAG)
  Cardinalidade: 3 | Nulls: 0.0%
  Valores frequentes:
    'CMV': 379,429 (97.3%)
    'NET': 9,003 (2.3%)
    'DTH': 1,509 (0.4%)
  Sugestoes:
    -> TAXA_PROD_[VALOR]
    -> FLAG_PROD_[VALOR]
    -> QTD_PROD_DISTINTOS

flag_mig2 (CATEGORICA_FLAG)
  Cardinalidade: 3 | Nulls: 32.45%
  Valores frequentes:
    'Aquisição': 134,116 (34.4%)
    'PRE': 128,855 (33.0%)
    'None': 126,524 (32.4%)
    'FLEX': 446 (0.1%)
  Sugestoes:
    -> TAXA_flag_mig2_[VALOR]
    -> FLAG_flag_mig2_[VALOR]
    -> QTD_flag_mig2_DISTINTOS

STATUSRF (CATEGORICA_FLAG)
  Cardinalidade: 6 | Nulls: 0.4%
  Valores frequentes:
    'REGULAR': 384,752 (98.7%)
    'PENDENTE DE REGULARIZACAO': 3,116 (0.8%)
    'None': 1,567 (0.4%)
    'SUSPENSA': 247 (0.1%)
    'TITULAR FALECIDO': 238 (0.1%)
  Sugestoes:
    -> TAXA_STATUSRF_[VALOR]
    -> FLAG_STATUSRF_[VALOR]
    -> QTD_STATUSRF_DISTINTOS

var_18 (CATEGORICA_FLAG)
  Cardinalidade: 1 | Nulls: 81.02%
  Valores fr

In [18]:
print("DETALHAMENTO - COLUNAS NUMERICAS")
print("="*80)

def fmt(v):
    return f"{v:.2f}" if isinstance(v, (int, float)) else "N/A"

numericas_com_stats = [
    m for m in metadados_enriquecidos
    if m['tipo_variavel'] == 'NUMERICA_VALOR' and m.get('percentis')
]

for meta in numericas_com_stats:
    p = meta['percentis']
    print(f"\n{meta['coluna']}")

    if p.get('min') is not None and p.get('max') is not None:
        print(f"  Range: [{fmt(p.get('min'))} -> {fmt(p.get('max'))}]")

    if p.get('media') is not None:
        print(f"  Media: {fmt(p.get('media'))} | Mediana: {fmt(p.get('p50'))}")

    if p.get('p25') is not None:
        print(
            f"  P25={fmt(p.get('p25'))} | "
            f"P75={fmt(p.get('p75'))} | "
            f"P95={fmt(p.get('p95'))}"
        )

    print("  Sugestoes FLAGS:")
    if p.get('p95') is not None:
        print(f"    -> FLAG_ALTO: valor > {fmt(p.get('p95'))} (P95)")
    if p.get('p25') is not None:
        print(f"    -> FLAG_BAIXO: valor < {fmt(p.get('p25'))} (P25)")


StatementMeta(, f62946fa-a604-4d89-a172-4db426cf81a6, 20, Finished, Available, Finished)

DETALHAMENTO - COLUNAS NUMERICAS

SAFRA
  Range: [202410.00 -> 202503.00]
  Media: 202456.16 | Mediana: 202412.00
  P25=202411.00 | P75=202502.00 | P95=202503.00
  Sugestoes FLAGS:
    -> FLAG_ALTO: valor > 202503.00 (P95)
    -> FLAG_BAIXO: valor < 202411.00 (P25)

FLAG_INSTALACAO
  Media: 0.69 | Mediana: 1.00
  P25=0.00 | P75=1.00 | P95=1.00
  Sugestoes FLAGS:
    -> FLAG_ALTO: valor > 1.00 (P95)
    -> FLAG_BAIXO: valor < 0.00 (P25)

FPD
  Media: 0.21 | Mediana: N/A
  P25=0.00 | P75=0.00 | P95=1.00
  Sugestoes FLAGS:
    -> FLAG_ALTO: valor > 1.00 (P95)
    -> FLAG_BAIXO: valor < 0.00 (P25)

var_03
  Range: [1.00 -> 100.00]
  Media: 32.77 | Mediana: 33.00
  P25=10.00 | P75=38.00 | P95=96.00
  Sugestoes FLAGS:
    -> FLAG_ALTO: valor > 96.00 (P95)
    -> FLAG_BAIXO: valor < 10.00 (P25)

var_02
  Range: [212.00 -> 992225.00]
  Media: 524265.97 | Mediana: 514320.00
  P25=411010.00 | P75=715610.00 | P95=821105.00
  Sugestoes FLAGS:
    -> FLAG_ALTO: valor > 821105.00 (P95)
    -> FLAG_B

In [19]:
print("EXPORTANDO METADADOS ENRIQUECIDOS")
print("="*80)

def clean_for_json(obj):
    if isinstance(obj, dict): return {k: clean_for_json(v) for k, v in obj.items()}
    elif isinstance(obj, list): return [clean_for_json(v) for v in obj]
    elif isinstance(obj, float): return None if obj != obj else round(obj, 4)
    else: return obj

metadados_json = clean_for_json(metadados_enriquecidos)

export_completo = {
    'tabela': f"{NOME_SCHEMA}.{NOME_TABELA}",
    'data_geracao': TIMESTAMP_EXECUCAO,
    'total_registros': total_registros,
    'total_colunas': total_colunas,
    'colunas_utilizaveis_book': len(colunas_book),
    'resumo_tipos': dict(contagem_tipos),
    'metadados_colunas': metadados_json
}

# ## DESCOMENTAR PARA SALVAR O JSON ###
# base_path = f"{PATH_OUTPUT}"
# os.makedirs(base_path, exist_ok=True)

# json_path = f"{base_path}/metadados_enriquecidos_{NOME_TABELA}.json"

# with open(json_path, "w", encoding="utf-8") as f:
#     json.dump(export_completo, f, ensure_ascii=False, indent=2)

# print(f"JSON salvo no Lakehouse em: {json_path}")
print("\n" + "="*80)
print("METADADOS PRONTOS PARA USAR NO GERADOR DE BOOK!")
print("="*80)

StatementMeta(, f62946fa-a604-4d89-a172-4db426cf81a6, 21, Finished, Available, Finished)

EXPORTANDO METADADOS ENRIQUECIDOS

METADADOS PRONTOS PARA USAR NO GERADOR DE BOOK!


In [20]:
print("PREVIA DO JSON")
print("="*80)
colunas_preview = [m for m in metadados_json if m['usar_no_book']][:3]
print(json.dumps(colunas_preview, indent=2, ensure_ascii=False))

StatementMeta(, f62946fa-a604-4d89-a172-4db426cf81a6, 22, Finished, Available, Finished)

PREVIA DO JSON
[
  {
    "coluna": "SAFRA",
    "tipo_spark": "IntegerType()",
    "tipo_variavel": "NUMERICA_VALOR",
    "cardinalidade": 6,
    "pct_null": 0.0,
    "qualidade_modelo": "BOA",
    "usar_no_book": true,
    "agregacoes_sugeridas": [
      "SUM",
      "AVG",
      "MAX",
      "MIN",
      "STDDEV",
      "COUNT"
    ],
    "valores_frequentes": [],
    "percentis": {
      "min": 202410.0,
      "p25": 202411.0,
      "p50": 202412.0,
      "p75": 202502.0,
      "p95": 202503.0,
      "p99": 202503.0,
      "max": 202503.0,
      "media": 202456.1566,
      "desvio_padrao": 45.506
    },
    "sugestoes_variaveis": [
      "VLR_SAFRA_TOTAL",
      "VLR_SAFRA_MEDIO",
      "VLR_SAFRA_MAX",
      "QTD_SAFRA"
    ]
  },
  {
    "coluna": "FLAG_INSTALACAO",
    "tipo_spark": "IntegerType()",
    "tipo_variavel": "NUMERICA_VALOR",
    "cardinalidade": 2,
    "pct_null": 0.0,
    "qualidade_modelo": "BOA",
    "usar_no_book": true,
    "agregacoes_sugeridas": [
      "SUM",

In [21]:
print("LIMPEZA DE CACHE")
df.unpersist()
print("Cache liberado")
print(f"\nAnalise da tabela '{NOME_SCHEMA}.{NOME_TABELA}' finalizada!")
print(f"Execucao concluida em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

StatementMeta(, f62946fa-a604-4d89-a172-4db426cf81a6, 23, Finished, Available, Finished)

LIMPEZA DE CACHE
Cache liberado

Analise da tabela 'rawdata.dados_cadastrais' finalizada!
Execucao concluida em: 2026-01-22 23:11:13
